In [19]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from feedback_grape.fgrape import optimize_pulse, evaluate_on_longer_time
from helpers import (
    init_fgrape_protocol,
    test_implementations,
    experiments_format,
    state_types,
)
from tqdm import tqdm
import jax
import numpy as np
from library.utils.FgResult_to_dict import FgResult_to_dict
import json

test_implementations()

In [ ]:
# Training and evaluation parameters
dir_ = "./12_results"

training_params = {
    "N_samples": 1, # Number of random initial states to sample
    "samples_offset": 0, # Offset for sample indexing
    "N_training_iterations": 10000, # Number of training iterations
    "learning_rate": 0.01, # Learning rate
    "convergence_threshold": 1e-6,
    "batch_size": 8,
    "eval_batch_size": 128,
    "evaluation_time_steps": 20,
}
N_parallel_threads = 1 # Number of parallel threads for training

# Physical Parameters to test

#num_time_steps : Number of time steps in the control pulse
#lut_memory : Amount of measurements the LUT remembers (depth of LUT)
#reward_weights : Weights for the reward at each time step. Default only weights last timestep [0, 0, ... 0, 1]
#N_qubits : Number of parallel chains to simulate
gamma_p, gamma_m = 0.0001, 0.05336699231206307
experiments = [ # (timesteps, lut_memory, reward_weights, N_qubits, N_meas, gamma_p, gamma_m, training_state_type, evaluation_state_type)
    (3, 2, [0,0,1], 3, 2, gamma_p, gamma_m, "bloch", "bloch"),
    (3, 2, [0,1,1], 3, 2, gamma_p, gamma_m, "bloch", "bloch"),
    (3, 2, [1,1,1], 3, 2, gamma_p, gamma_m, "bloch", "bloch"),
    #for gamma_m in np.logspace(np.log10(0.0001), np.log10(0.1), 12).tolist()
    #for gamma_p in np.logspace(np.log10(0.0001), np.log10(0.1), 12).tolist()
]

# Check if experiment parameters are in specified format
for params in experiments:
    assert len(params) == len(experiments_format), "Experiment parameters length mismatch."
    for i, (param, (name, expected_type, req_grape, req_lut, req_rnn)) in enumerate(zip(params, experiments_format)):
        assert type(name) == str, "Parameter name must be a string."
        assert type(req_grape) == bool, "Requirement flags must be boolean."
        assert type(req_lut) == bool, "Requirement flags must be boolean."
        assert type(req_rnn) == bool, "Requirement flags must be boolean."
        assert type(param) == expected_type, f"Parameter '{name}' at index {i} must be of type {expected_type.__name__}."
        if type(param) == list:
            for j, item in enumerate(param):
                assert type(item) == int and 0 <= item <= 9, f"Item at index {j} in parameter '{name}' must be of type int and between 0 and 9."

# Check if directory structure exists, create if not
for dir__ in [dir_, f"{dir_}/models", f"{dir_}/training_params", f"{dir_}/eval"]:
    os.makedirs(dir__, exist_ok=True)

# Generate parameter-based file paths
def generate_param_paths(dir_, params, model_type, s):
    assert model_type in ["grape", "lut", "rnn"], "Invalid model type."

    parts = []
    for (name, expected_type, req_grape, req_lut, req_rnn), param in zip(experiments_format, params):
        required = (model_type == "grape" and req_grape) or (model_type == "lut" and req_lut) or (model_type == "rnn" and req_rnn)
        if required:
            if expected_type == list:
                param_str = "".join(str(x) for x in param)
            else:
                param_str = str(param)
            parts.append(f"{name}={param_str}")

    pstr = f"{model_type}_" + "_".join(parts)
    pstr += f"_s={s}"
    
    return (
        f"{dir_}/models/{pstr}.json",
        f"{dir_}/training_params/{pstr}.json",
        f"{dir_}/eval/{pstr}.json",
    )

In [21]:
from concurrent.futures import ThreadPoolExecutor

def run_experiment(params):
    (
        num_time_steps,
        lut_memory,
        reward_weights,
        N_qubits,
        N_meas,
        gamma_p,
        gamma_m,
        training_state_type,
        evaluation_state_type
    ) = params
    
    print(f"Evaluating {params}")
    training_state = lambda key: state_types[training_state_type](key, N_qubits)
    evaluation_state = lambda key: state_types[evaluation_state_type](key, N_qubits)

    for s in tqdm(range(training_params["samples_offset"], training_params["N_samples"] + training_params["samples_offset"])):
        path1, path2, path3 = generate_param_paths(dir_, params, "rnn", s)
        #if os.path.exists(path1):
        #    print(f"Skipping existing experiment for params {params} and seed {s}.")
        #    continue

        with open(path2, "w") as f:
            json.dump(training_params, f)

        system_params = init_fgrape_protocol(
            jax.random.PRNGKey(s),
            N_qubits,
            N_meas,
            gamma_p,
            gamma_m,
        )

        # Train LUT
        result = optimize_pulse(
            U_0=training_state,
            C_target=training_state,
            system_params=system_params,
            num_time_steps=num_time_steps,
            #lut_depth=lut_memory,
            reward_weights=reward_weights,
            mode="nn",
            goal="fidelity",
            max_iter=training_params["N_training_iterations"],
            convergence_threshold=training_params["convergence_threshold"],
            learning_rate=training_params["learning_rate"],
            evo_type="density",
            batch_size=training_params["batch_size"],
            eval_batch_size=1, # This evaluation is discarded
        )

        with open(path1, "w") as f:
            json.dump(FgResult_to_dict(result), f)

        eval_result = evaluate_on_longer_time( # Evaluate on longer time and choose best LUT accordingly
            U_0 = evaluation_state,
            C_target = evaluation_state,
            system_params = system_params,
            optimized_trainable_parameters = result.optimized_trainable_parameters,
            num_time_steps = training_params["evaluation_time_steps"],
            evo_type = "density",
            goal = "fidelity",
            eval_batch_size = training_params["eval_batch_size"],
            mode = "nn",
        )

        with open(path3, "w") as f:
            json.dump(FgResult_to_dict(eval_result), f)
            del eval_result  # Free memory
        del result  # Free memory
        del system_params  # Free memory

# Run experiments in parallel
if N_parallel_threads > 1:
    with ThreadPoolExecutor(max_workers=N_parallel_threads) as executor:
        executor.map(run_experiment, experiments)
else:
    for params in experiments:
        run_experiment(params)

# Play a sound when done
os.system('say "done."')

Evaluating (3, 2, [0, 0, 1], 3, 2, 0.0001, 0.05336699231206307, 'bloch', 'bloch')


100%|██████████| 1/1 [00:20<00:00, 20.74s/it]


Evaluating (3, 2, [0, 1, 1], 3, 2, 0.0001, 0.05336699231206307, 'bloch', 'bloch')


100%|██████████| 1/1 [00:21<00:00, 21.80s/it]


Evaluating (3, 2, [1, 1, 1], 3, 2, 0.0001, 0.05336699231206307, 'bloch', 'bloch')


100%|██████████| 1/1 [00:20<00:00, 20.97s/it]


0